# 🧠 Symbolic Transformer Training

Train a tiny transformer to predict next symbols in First-Order Logic formulas.

**What this does:**
- Generates synthetic FOL training data
- Trains a small transformer model (566K - 19.6M parameters)
- Learns syntax rules like: `∀` → must be followed by `VAR`

**Quick Start:** Run cells 1-4 in order. Training takes ~30s-90s/epoch on GPU.

---

## 1️⃣ Setup Environment
Clone the repository and install dependencies.

In [1]:
#@title 1. Setup Environment { display-mode: "form" }
#@markdown Run this cell first to set up the environment.

import os

# Check GPU
print("🔍 Checking GPU availability...")
!nvidia-smi --query-gpu=name,memory.total --format=csv,noheader 2>/dev/null || echo "⚠️ No GPU detected - training will be slower"

# Clone repository
print("\n📦 Cloning Symbolic-Transformers repository...")
%cd /content
if not os.path.exists('Symbolic-Transformers'):
    !git clone -q https://github.com/tripptytrip/Symbolic-Transformers.git
    print("✓ Repository cloned")
else:
    !cd Symbolic-Transformers && git stash && git pull -q
    print("✓ Repository updated")

%cd /content/Symbolic-Transformers

# Install dependencies
print("\n📚 Installing dependencies...")
!pip -q install numpy scipy pandas tqdm rich tensorboard
print("✓ Dependencies installed")

# Verify vocabulary
print("\n🔤 Verifying vocabulary...")
!python -c "from utils.vocabulary import Vocabulary; v = Vocabulary('unified_vocabulary.json'); print(f'✓ Vocabulary loaded: {v.vocab_size} tokens')"

print("\n" + "="*50)
print("✅ Setup complete! Proceed to Step 2.")
print("="*50)

🔍 Checking GPU availability...
NVIDIA A100-SXM4-80GB, 81920 MiB

📦 Cloning Symbolic-Transformers repository...
/content
✓ Repository cloned
/content/Symbolic-Transformers

📚 Installing dependencies...
✓ Dependencies installed

🔤 Verifying vocabulary...
✓ Vocabulary loaded: 662 tokens
  - Numerals: 0-624
  - Symbols: 625-661
  - Compositional: ['VAR', 'CONST', 'PRED', 'FUNC', 'SORT']
✓ Vocabulary loaded: 662 tokens

✅ Setup complete! Proceed to Step 2.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2️⃣ Configure Training
Adjust the settings below, then run the cell to apply them.

In [2]:
#@title 2. Training Configuration { display-mode: "form" }

#@markdown ### 🏗️ Model Size
model_size = "tiny" #@param ["tiny", "small", "base"]
#@markdown - **tiny**: 566K params (~2.2MB) - Fast training, good for experiments
#@markdown - **small**: 3.5M params (~14MB) - Better accuracy, moderate training time
#@markdown - **base**: 19.6M params (~78MB) - Best accuracy, longest training

#@markdown ---
#@markdown ### 📊 Dataset Size
num_train_formulas = 50000 #@param {type:"slider", min:1000, max:50000, step:1000}
#@markdown Number of unique FOL formulas to generate for training.
#@markdown - 1000-3000: Quick experiments
#@markdown - 5000-10000: Standard training
#@markdown - 20000-50000: Large-scale training (recommended for small/base models)

num_val_formulas = 5000 #@param {type:"slider", min:100, max:5000, step:100}
#@markdown Number of formulas for validation (typically 10-20% of training).

#@markdown ---
#@markdown ### 🧬 Data Generator
use_advanced_generator = True #@param {type:"boolean"}
#@markdown **Advanced generator** adds:
#@markdown - Function symbols: `P(f(x), g(y))` instead of just `P(x, y)`
#@markdown - Fixed signatures: `PRED_5` is *always* arity 2 (model learns consistency)
#@markdown - Horn clauses: `(A ∧ B ∧ C) → D` (common logic programming pattern)
#@markdown - Vacuous quantification: `∀x P(y)` (tests scope understanding)

#@markdown ---
#@markdown ### ⚙️ Training Parameters
num_epochs = 500 #@param {type:"slider", min:10, max:500, step:10}
#@markdown Number of training epochs.
#@markdown - 10-30: Quick experiments
#@markdown - 50-100: Standard training
#@markdown - 100-200: Train to convergence (watch for overfitting!)

batch_size = 32 #@param [32, 64, 128, 256] {type:"raw"}
#@markdown Larger batches = faster training but more memory.

#@markdown ---
#@markdown ### 💾 Resume from Checkpoint
resume_training = False #@param {type:"boolean"}
#@markdown Resume from the last saved checkpoint.

# Store configuration
config = {
    'model_size': model_size,
    'num_train_formulas': num_train_formulas,
    'num_val_formulas': num_val_formulas,
    'num_test_formulas': max(100, num_val_formulas // 2),
    'num_epochs': num_epochs,
    'batch_size': batch_size,
    'resume': resume_training,
    'use_advanced_generator': use_advanced_generator
}

# Display configuration summary
print("="*50)
print("📋 TRAINING CONFIGURATION")
print("="*50)

model_params = {'tiny': '566K', 'small': '3.5M', 'base': '19.6M'}
model_size_mb = {'tiny': '~2.2MB', 'small': '~14MB', 'base': '~78MB'}

print(f"\n🏗️  Model:     {model_size} ({model_params[model_size]} parameters, {model_size_mb[model_size]})")
print(f"📊 Dataset:   {num_train_formulas} train / {num_val_formulas} val formulas")
print(f"🧬 Generator: {'Advanced (functions, fixed signatures, Horn clauses)' if use_advanced_generator else 'Basic'}")
print(f"⚙️  Training:  {num_epochs} epochs, batch size {batch_size}")
print(f"🔄 Resume:    {'Yes' if resume_training else 'No (fresh start)'}")

# Estimate training time
samples_per_formula = 15 if use_advanced_generator else 14
total_samples = num_train_formulas * samples_per_formula
batches_per_epoch = total_samples // batch_size
time_per_batch = {'tiny': 0.008, 'small': 0.020, 'base': 0.040}  # seconds on A100
est_epoch_time = batches_per_epoch * time_per_batch[model_size]
est_total_time = est_epoch_time * num_epochs / 60

print(f"\n⏱️  Estimated training time: ~{est_total_time:.0f} minutes on A100")
print(f"   ({est_epoch_time:.0f}s per epoch, {batches_per_epoch} batches)")

# Recommendations
if model_size in ['small', 'base'] and num_train_formulas < 10000:
    print(f"\n💡 TIP: {model_size} model benefits from more data.")
    print(f"   Consider increasing to 20000+ formulas.")

if num_epochs > 100 and not use_advanced_generator:
    print(f"\n⚠️  WARNING: High epochs ({num_epochs}) with basic generator.")
    print(f"   Risk of overfitting! Consider:")
    print(f"   - Enabling advanced generator for richer data")
    print(f"   - Or reducing epochs to 50-100")

print("\n" + "="*50)
print("✅ Configuration saved! Proceed to Step 3.")
print("="*50)

📋 TRAINING CONFIGURATION

🏗️  Model:     tiny (566K parameters, ~2.2MB)
📊 Dataset:   50000 train / 5000 val formulas
🧬 Generator: Advanced (functions, fixed signatures, Horn clauses)
⚙️  Training:  500 epochs, batch size 32
🔄 Resume:    No (fresh start)

⏱️  Estimated training time: ~1562 minutes on A100
   (187s per epoch, 23437 batches)

✅ Configuration saved! Proceed to Step 3.


## 3️⃣ Generate Training Data
Create synthetic First-Order Logic formulas for training.

In [3]:
#@title 3. Generate Training Data { display-mode: "form" }
#@markdown This generates synthetic FOL formulas.
#@markdown
#@markdown **Basic generator examples:**
#@markdown - `∀x₁ (P₅(x₁) → Q₂(x₁))`
#@markdown - `∃x₀ ∃x₁ (R₃(x₀, x₁) ∧ P₁(x₀))`
#@markdown
#@markdown **Advanced generator adds:**
#@markdown - `∀x P(f(x), g(x, y))` (function symbols)
#@markdown - `(A ∧ B ∧ C) → D` (Horn clauses)
#@markdown - Fixed predicate arities across all formulas

import os
os.chdir('/content/Symbolic-Transformers')

print("🔄 Generating training data...")
print(f"   Train: {config['num_train_formulas']} formulas")
print(f"   Val:   {config['num_val_formulas']} formulas")
print(f"   Test:  {config['num_test_formulas']} formulas")
print(f"   Generator: {'Advanced' if config['use_advanced_generator'] else 'Basic'}")
print()

import sys
sys.path.insert(0, '/content/Symbolic-Transformers')

if config['use_advanced_generator']:
    # Use advanced generator with functions, fixed signatures, Horn clauses
    from data.advanced_generator import generate_advanced_training_data

    generate_advanced_training_data(
        vocab_path="unified_vocabulary.json",
        output_dir="datasets/fol_next_symbol",
        n_train=config['num_train_formulas'],
        n_val=config['num_val_formulas'],
        n_test=config['num_test_formulas'],
    )
else:
    # Use basic generator
    from data.dataset_generator import generate_training_data

    generate_training_data(
        vocab_path="unified_vocabulary.json",
        output_dir="datasets/fol_next_symbol",
        n_train=config['num_train_formulas'],
        n_val=config['num_val_formulas'],
        n_test=config['num_test_formulas'],
    )

# Show dataset stats
print("\n📁 Dataset files:")
!ls -lh datasets/fol_next_symbol/*.json

print("\n" + "="*50)
print("✅ Data generated! Proceed to Step 4.")
print("="*50)

🔄 Generating training data...
   Train: 50000 formulas
   Val:   5000 formulas
   Test:  2500 formulas
   Generator: Advanced

ADVANCED FOL DATASET GENERATION
✓ Vocabulary loaded: 662 tokens
  - Numerals: 0-624
  - Symbols: 625-661
  - Compositional: ['VAR', 'CONST', 'PRED', 'FUNC', 'SORT']

ℹ️ Fixed Signatures:
   Predicates: {0: 2, 1: 1, 2: 1, 3: 1, 4: 2, 5: 2, 6: 2, 7: 1, 8: 2, 9: 1}
   Functions:  {0: 1, 1: 1, 2: 1, 3: 1}

Generating train set (50000 formulas)...
  Complexity 1: 10000 formulas
  Complexity 2: 20000 formulas
  Complexity 3: 15000 formulas
  Complexity 4: 5000 formulas

Generating val set (5000 formulas)...
  Complexity 1: 1000 formulas
  Complexity 2: 2000 formulas
  Complexity 3: 1500 formulas
  Complexity 4: 500 formulas

Generating test set (2500 formulas)...
  Complexity 1: 500 formulas
  Complexity 2: 1000 formulas
  Complexity 3: 750 formulas
  Complexity 4: 250 formulas
✓ Saved 1673700 samples to datasets/fol_next_symbol/train.json
✓ Saved 165213 samples to d

## 4️⃣ Train the Model
Start training! Watch the loss decrease over epochs.

In [ ]:
#@title 4. Train Model { display-mode: "form" }
import os
os.chdir('/content/Symbolic-Transformers')

print("="*60)
print("🚀 STARTING TRAINING")
print("="*60)
# Debug print to verify config
print(f"Model: {config['model_size']} | Epochs: {config['num_epochs']} | Batch Size: {config['batch_size']}")

if config['resume']:
    print("Resuming from last checkpoint...")
print("="*60 + "\n")

# FIX: Added --batch-size to the command string
cmd = f"python training/train.py --model-size {config['model_size']} --num-epochs {config['num_epochs']} --batch-size {config['batch_size']}"

if config['resume']:
    cmd += " --resume"

# Run training
!{cmd}

print("\n" + "="*60)
print("✅ TRAINING COMPLETE!")
print("="*60)
print("\n📁 Saved checkpoints:")
!ls -lh checkpoints/*.pt 2>/dev/null | tail -5
print("\nBest model saved to: checkpoints/best_model.pt")

🚀 STARTING TRAINING
Model: tiny | Epochs: 500 | Batch Size: 32

✓ Vocabulary loaded: 662 tokens
  - Numerals: 0-624
  - Symbols: 625-661
  - Compositional: ['VAR', 'CONST', 'PRED', 'FUNC', 'SORT']

Loading datasets...
✓ Loaded 1673700 samples from datasets/fol_next_symbol/train.json
✓ Loaded 165213 samples from datasets/fol_next_symbol/val.json

Creating model...
✓ Created tiny model with 566,934 parameters
✓ Using device: cuda
✓ GPU: NVIDIA A100-SXM4-80GB

TRAINING START
Model: tiny
Vocab size: 662
Batch size: 32
Epochs: 500
Device: cuda

Epoch 1/500
  Batch 100/52304 | Loss: 6.5070 | LR: 5.00e-06
  Batch 200/52304 | Loss: 6.4259 | LR: 1.00e-05
  Batch 300/52304 | Loss: 6.2977 | LR: 1.50e-05
  Batch 400/52304 | Loss: 6.1273 | LR: 2.00e-05
  Batch 500/52304 | Loss: 5.9391 | LR: 2.50e-05
  Batch 600/52304 | Loss: 5.7297 | LR: 3.00e-05
  Batch 700/52304 | Loss: 5.5057 | LR: 3.50e-05
  Batch 800/52304 | Loss: 5.2739 | LR: 4.00e-05
  Batch 900/52304 | Loss: 5.0467 | LR: 4.50e-05
  Batch 10

## 5️⃣ Evaluate Model (Optional)
Run evaluation on the test set to see accuracy metrics.

In [ ]:
#@title 5. Evaluate Model { display-mode: "form" }
#@markdown Run evaluation on the test set.

import os
os.chdir('/content/Symbolic-Transformers')

print("📊 Evaluating model on test set...\n")

!python evaluate_model.py \
    --checkpoint checkpoints/best_model.pt \
    --test-data datasets/fol_next_symbol/test.json

📊 Evaluating model on test set...

Using device: cuda
Vocabulary size: 662
Loading data from: datasets/fol_next_symbol/test.json
Test samples: 81694
Loading checkpoint: checkpoints/best_model.pt
✓ Created tiny model with 566,934 parameters
✓ Loaded model from epoch 5
✓ Best val loss: 0.7796

STARTING EVALUATION
Evaluating: 100% 639/639 [00:02<00:00, 220.07it/s]

RESULTS
Top-1 Accuracy:  63.43% (63.4%)
Top-5 Accuracy:  91.09% (91.1%)
Top-10 Accuracy: 95.50% (95.5%)

Improvement over random (0.15%): 419.9x


## 6️⃣ Download Trained Model (Optional)
Download the trained model checkpoint to your local machine.

In [ ]:
#@title 6. Download Model { display-mode: "form" }
#@markdown Download the best model checkpoint.

from google.colab import files
import os

checkpoint_path = '/content/Symbolic-Transformers/checkpoints/best_model.pt'

if os.path.exists(checkpoint_path):
    print(f"📦 Preparing download...")
    file_size = os.path.getsize(checkpoint_path) / (1024 * 1024)
    print(f"   File: best_model.pt ({file_size:.1f} MB)")
    print(f"   Model: {config['model_size']}")
    print()
    files.download(checkpoint_path)
else:
    print("❌ No checkpoint found. Run training first (Step 4).")

📦 Preparing download...
   File: best_model.pt (6.8 MB)
   Model: tiny



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 7️⃣ Interactive Demo (Optional)
Try the model interactively - type tokens and see predictions!

In [ ]:
#@title 7. Quick Demo { display-mode: "form" }
#@markdown See the model's predictions for a sample input.

import torch
import sys
import os

# Ensure we can import from the repository
sys.path.insert(0, '/content/Symbolic-Transformers')

from utils.vocabulary import Vocabulary
# FIX: Import create_model instead of SymbolicTransformer and get_model_config
from models.transformer import create_model

# Load model
print("🔄 Loading model...")
vocab = Vocabulary('unified_vocabulary.json')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if not os.path.exists('checkpoints/best_model.pt'):
    print("❌ Error: 'checkpoints/best_model.pt' not found. Run training first.")
else:
    checkpoint = torch.load('checkpoints/best_model.pt', map_location=device)
    config = checkpoint['config']

    # FIX: Use create_model factory function
    model = create_model(vocab_size=vocab.vocab_size, model_size=config['model_size'])
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.to(device)
    model.eval()

    print(f"✓ Loaded {config['model_size']} model from epoch {checkpoint['epoch']}")
    if 'best_val_loss' in checkpoint:
        print(f"  Val loss: {checkpoint['best_val_loss']:.4f}\n")

    # Demo predictions
    def predict_next(tokens, top_k=5):
        """Predict next token given a sequence."""
        # Handle both token strings and direct IDs if needed
        token_ids = []
        for t in tokens:
            if t in vocab.label_to_id:
                token_ids.append(vocab.encode_label(t))
            else:
                try:
                    token_ids.append(int(t))
                except ValueError:
                    print(f"Warning: Skipping unknown token '{t}'")
                    continue

        x = torch.tensor([token_ids], device=device, dtype=torch.long)

        with torch.no_grad():
            logits = model(x)
            probs = torch.softmax(logits[0, -1], dim=-1)
            top_probs, top_ids = torch.topk(probs, top_k)

        print(f"Input: {' '.join(tokens)}")
        print(f"Top {top_k} predictions:")
        for i, (prob, tid) in enumerate(zip(top_probs, top_ids)):
            label = vocab.decode_id(tid.item())
            bar = '█' * int(prob * 30)
            print(f"  {i+1}. {label:12s} {prob*100:5.1f}% {bar}")
        print()

    # Show example predictions
    print("="*50)
    print("📊 EXAMPLE PREDICTIONS")
    print("="*50 + "\n")

    # After FORALL, should predict VAR with high confidence
    predict_next(['FORALL'])

    # After FORALL VAR, should predict a numeral
    predict_next(['FORALL', 'VAR'])

    # After a complete variable binding
    predict_next(['FORALL', 'VAR', '1'])

    # After predicate (should predict LPAREN)
    predict_next(['PRED', '3'])

    print("\n💡 The model learned FOL syntax rules!")

🔄 Loading model...
✓ Vocabulary loaded: 662 tokens
  - Numerals: 0-624
  - Symbols: 625-661
  - Compositional: ['VAR', 'CONST', 'PRED', 'FUNC', 'SORT']
✓ Created tiny model with 566,934 parameters
✓ Loaded tiny model from epoch 5
  Val loss: 0.7796

📊 EXAMPLE PREDICTIONS

Input: FORALL
Top 5 predictions:
  1. VAR          100.0% █████████████████████████████
  2. FUNC           0.0% 
  3. CONST          0.0% 
  4. EXISTS         0.0% 
  5. PRED           0.0% 

Input: FORALL VAR
Top 5 predictions:
  1. NUM_005       17.2% █████
  2. NUM_007       15.6% ████
  3. NUM_006       14.7% ████
  4. NUM_002       13.9% ████
  5. NUM_001       13.4% ████

Input: FORALL VAR 1
Top 5 predictions:
  1. LPAREN        45.0% █████████████
  2. EXISTS        15.7% ████
  3. NOT           15.2% ████
  4. FORALL        13.9% ████
  5. PRED           7.9% ██

Input: PRED 3
Top 5 predictions:
  1. LPAREN       100.0% █████████████████████████████
  2. NOT            0.0% 
  3. EXISTS         0.0% 
  4. FOR

---

## 📖 Quick Reference

### Model Sizes
| Size | Parameters | File Size | Time/Epoch (A100) | Capacity |
|------|-----------|-----------|-------------------|----------|
| tiny | 566K | ~2.2MB | ~30s | Good for <10K formulas |
| small | 3.5M | ~14MB | ~90s | Good for 10-50K formulas |
| base | 19.6M | ~78MB | ~180s | Good for 50K+ formulas |

### Data Generator Comparison
| Feature | Basic | Advanced |
|---------|-------|----------|
| Predicates `P(x, y)` | ✓ | ✓ |
| Functions `f(x)` | ✗ | ✓ |
| Nested terms `P(f(g(x)))` | ✗ | ✓ |
| Fixed arities | ✗ | ✓ |
| Horn clauses | ✗ | ✓ |
| Vacuous quantification | ✗ | ✓ |

### Recommended Configurations

**Quick Test (5-10 min):**
- Model: tiny
- Data: 1000 formulas (basic)
- Epochs: 20

**Standard Training (30-60 min):**
- Model: small
- Data: 10000 formulas (advanced)
- Epochs: 50-100

**Best Results (2+ hours):**
- Model: base
- Data: 30000+ formulas (advanced)
- Epochs: 100-200 (watch val loss!)

### Interpreting Results
- **Val Loss > 1.5**: Model is still learning basic patterns
- **Val Loss 1.0-1.5**: Good progress, learning syntax rules
- **Val Loss 0.85-1.0**: Excellent, model understands FOL structure
- **Val Loss < 0.85**: Very good, approaching optimal

### ⚠️ Overfitting Warning Signs
- Train loss keeps dropping but val loss stops improving
- Val loss starts **increasing** while train loss decreases
- Gap between train and val loss > 0.1

**If overfitting:**
1. Stop training and use the checkpoint with lowest val loss
2. Generate more training data
3. Enable advanced generator for richer patterns

### Files Created
- `checkpoints/best_model.pt` - Best performing model
- `checkpoints/checkpoint_epoch_N.pt` - Periodic checkpoints
- `datasets/fol_next_symbol/` - Training data

# Task
Read the content of `evaluate_model.py` to understand how it loads the model and processes the test data, specifically looking for how it handles batching and if it accepts a `batch_size` argument.

## Inspect Evaluate Model Script

### Subtask:
Read the content of `evaluate_model.py` to understand its arguments and data processing logic.


**Reasoning**:
Read the content of the `evaluate_model.py` script to understand its implementation details as requested.



In [ ]:
import os

# Change directory to the repository
os.chdir('/content/Symbolic-Transformers')

# Display the content of evaluate_model.py
!cat evaluate_model.py

#!/usr/bin/env python3
"""
Evaluate trained FOL model.
Standalone script with all imports properly configured.
"""

import os
import sys
import torch
import torch.nn.functional as F
from pathlib import Path
import json

# Add fol_transformer to path
sys.path.insert(0, os.path.join(os.path.dirname(__file__), 'fol_transformer'))

def load_vocabulary(vocab_path):
    """Load vocabulary."""
    with open(vocab_path, 'r', encoding='utf-8') as f:
        vocab = json.load(f)
    return vocab

def load_model_checkpoint(checkpoint_path, vocab_size, device='cpu'):
    """Load trained model from checkpoint."""
    from models.transformer import create_model
    
    checkpoint = torch.load(checkpoint_path, map_location=device)
    config = checkpoint['config']
    
    # Create model
    model = create_model(vocab_size, config['model_size'])
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    print(f"✓ Loaded model from epoch {checkpoint['epoch']}")
    print(f"✓ 

# Task
Rewrite `evaluate_model.py` to implement batch processing using `torch.utils.data.DataLoader` for faster evaluation and add command-line arguments (including `--batch-size`) using `argparse`.

```python
%%writefile evaluate_model.py
#!/usr/bin/env python3
"""
Evaluate trained FOL model with batch processing.
Optimized for speed using DataLoader.
"""

import os
import sys
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from pathlib import Path
import json
import argparse
from tqdm import tqdm

# Add fol_transformer to path
sys.path.insert(0, os.path.join(os.path.dirname(__file__), 'fol_transformer'))

def load_vocabulary(vocab_path):
    """Load vocabulary."""
    with open(vocab_path, 'r', encoding='utf-8') as f:
        vocab = json.load(f)
    return vocab

def load_model_checkpoint(checkpoint_path, vocab_size, device='cpu'):
    """Load trained model from checkpoint."""
    from models.transformer import create_model
    
    print(f"Loading checkpoint: {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path, map_location=device)
    config = checkpoint['config']
    
    # Create model
    model = create_model(vocab_size, config['model_size'])
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    model.eval()
    
    print(f"✓ Loaded {config['model_size']} model from epoch {checkpoint['epoch']}")
    if 'best_val_loss' in checkpoint:
        print(f"✓ Best val loss: {checkpoint['best_val_loss']:.4f}")
    
    return model

class FOLDataset(Dataset):
    """Dataset for FOL evaluation."""
    def __init__(self, data_path):
        with open(data_path, 'r') as f:
            data = json.load(f)
        
        # Handle different data formats
        if 'samples' in data:
            self.samples = data['samples']
        else:
            self.samples = data
            
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        sample = self.samples[idx]
        
        # Handle different key names
        if 'input_ids' in sample:
            context = sample['input_ids']
        elif 'context' in sample:
            context = sample['context']
        elif 'input' in sample:
            context = sample['input']
        else:
            raise ValueError(f"Unknown sample format: {sample.keys()}")
            
        target = sample['target']
        
        # Ensure context is a list of ints
        if not isinstance(context, list):
            context = context.tolist()
            
        return torch.tensor(context, dtype=torch.long), target

def collate_fn(batch):
    """Pad sequences in batch."""
    inputs, targets = zip(*batch)
    # Pad inputs with 0 (assuming 0 is PAD/ignored)
    padded_inputs = pad_sequence(inputs, batch_first=True, padding_value=0)
    return padded_inputs, torch.tensor(targets, dtype=torch.long)

def compute_metrics(model, dataloader, device='cpu'):
    """Compute accuracy metrics using batch processing."""
    correct_top1 = 0
    correct_top5 = 0
    correct_top10 = 0
    total = 0
    
    model.eval()
    print("Computing metrics...")
    
    with torch.no_grad():
        for inputs, targets in tqdm(dataloader, desc="Evaluating"):
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            # Forward pass
            logits = model(inputs)
            # Get predictions for the last token: [batch_size, vocab_size]
            next_token_logits = logits[:, -1, :]
            
            # Top-k predictions
            _, top_k_preds = torch.topk(next_token_logits, k=10, dim=1)
            
            # Check accuracy
            # top_k_preds: [batch_size, 10]
            # targets: [batch_size]
            
            # Top-1
            correct_top1 += (top_k_preds[:, 0] == targets).sum().item()
            
            # Top-5
            # unsqueeze targets to [batch_size, 1] for broadcasting
            targets_expanded = targets.unsqueeze(1)
            correct_top5 += (top_k_preds[:, :5] == targets_expanded).any(dim=1).sum().item()
            
            # Top-10
            correct_top10 += (top_k_preds[:, :10] == targets_expanded).any(dim=1).sum().item()
            
            total += targets.size(0)
            
    return {
        'top1': correct_top1 / total,
        'top5': correct_top5 / total,
        'top10': correct_top10 / total,
        'total': total
    }

def decode_id(vocab, token_id):
    """Decode token ID to label."""
    id_to_label = {int(k): v for k, v in vocab['id_to_label'].items()}
    return id_to_label.get(token_id, f"UNK_{token_id}")

def main():
    parser = argparse.ArgumentParser(description='Evaluate FOL Model')
    parser.add_argument('--checkpoint', type=str, default='checkpoints/best_model.pt',
                        help='Path to model checkpoint')
    parser.add_argument('--test-data', type=str, default='datasets/fol_next_symbol/test.json',
                        help='Path to test data JSON')
    parser.add_argument('--vocab', type=str, default='unified_vocabulary.json',
                        help='Path to vocabulary JSON')
    parser.add_argument('--batch-size', type=int, default=32,
                        help='Batch size for evaluation')
    parser.add_argument('--cpu', action='store_true', help='Force CPU usage')
    
    args = parser.parse_args()
    
    print("\n" + "="*60)
    print("EVALUATING TRAINED MODEL (BATCH MODE)")
    print("="*60)
    
    # Check files
    for path in [args.vocab, args.checkpoint, args.test_data]:
        if not Path(path).exists():
            print(f"❌ File not found: {path}")
            return

    # Setup device
    device = torch.device('cuda' if torch.cuda.is_available() and not args.cpu else 'cpu')
    print(f"Device: {device}")
    
    # Load vocabulary
    print("\nLoading vocabulary...")
    vocab = load_vocabulary(args.vocab)
    vocab_size = vocab['vocab_size']
    print(f"✓ Vocabulary size: {vocab_size}")
    
    # Load model
    print("\nLoading model...")
    model = load_model_checkpoint(args.checkpoint, vocab_size, device)
    
    # Load data
    print("\nLoading test data...")
    dataset = FOLDataset(args.test_data)
    dataloader = DataLoader(
        dataset,
        batch_size=args.batch_size,
        shuffle=False,
        collate_fn=collate_fn,
        num_workers=2 if str(device) == 'cuda' else 0
    )
    print(f"✓ Test samples: {len(dataset)}")
    print(f"✓ Batch size: {args.batch_size}")
    
    # Compute metrics
    print("\n" + "="*60)
    print("ACCURACY METRICS")
    print("="*60)
    
    metrics = compute_metrics(model, dataloader, device)
    
    print(f"\n✓ Top-1 Accuracy: {metrics['top1']:.2%}")
    print(f"  ({metrics['top1']*100:.1f}% chance of exact next symbol)")
    
    print(f"\n✓ Top-5 Accuracy: {metrics['top5']:.2%}")
    print(f"  ({metrics['top5']*100:.1f}% chance target in top 5)")
    
    print(f"\n✓ Top-10 Accuracy: {metrics['top10']:.2%}")
    print(f"  ({metrics['top10']*100:.1f}% chance target in top 10)")
    
    # Random baseline
    random_acc = 1.0 / vocab_size
    improvement = metrics['top1'] / random_acc
    print(f"\n📊 Improvement over random guessing:")
    print(f"   Random: {random_acc:.2%}")
    print(f"   Model:  {metrics['top1']:.2%}")
    print(f"   {improvement:.1f}× better!")
    
    print("\n" + "="*60)
    print("✅ Evaluation complete!")
    print("="*60)

if __name__ == "__main__":
    main()
```

## Optimize Evaluation Script

### Subtask:
Rewrite `evaluate_model.py` to implement batch processing using `torch.utils.data.DataLoader` for faster evaluation and add command-line arguments using `argparse`.


**Reasoning**:
Rewrite evaluate_model.py to implement batch processing and argument parsing as requested.



In [ ]:
%%writefile evaluate_model.py
#!/usr/bin/env python3
"""
Evaluate trained FOL model with batch processing.
Optimized for speed using DataLoader.
"""

import os
import sys
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from pathlib import Path
import json
import argparse
from tqdm import tqdm

# Add fol_transformer to path to ensure imports work
sys.path.insert(0, os.path.join(os.path.dirname(__file__), 'fol_transformer'))

class FOLDataset(Dataset):
    """Dataset for FOL formulas."""
    def __init__(self, samples):
        self.samples = samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        # Handle different formats
        if 'input_ids' in sample:
            context = sample['input_ids']
            target = sample['target']
        elif 'context' in sample:
            context = sample['context']
            target = sample['target']
        elif 'input' in sample and 'target' in sample:
            context = sample['input']
            target = sample['target']
        else:
            # Fallback or error
            context = []
            target = 0

        return torch.tensor(context, dtype=torch.long), torch.tensor(target, dtype=torch.long)

def collate_fn(batch):
    """Pad sequences in batch."""
    contexts, targets = zip(*batch)
    # Pad contexts to max length in this batch
    padded_contexts = pad_sequence(contexts, batch_first=True, padding_value=0)
    targets = torch.stack(targets)
    return padded_contexts, targets

def load_vocabulary(vocab_path):
    """Load vocabulary."""
    with open(vocab_path, 'r', encoding='utf-8') as f:
        vocab = json.load(f)
    return vocab

def load_model_checkpoint(checkpoint_path, vocab_size, device='cpu'):
    """Load trained model from checkpoint."""
    # Import here to avoid issues if paths aren't set up at module level
    from models.transformer import create_model

    print(f"Loading checkpoint: {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path, map_location=device)
    config = checkpoint['config']

    # Create model
    model = create_model(vocab_size, config['model_size'])
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    model.eval()

    print(f"✓ Loaded model from epoch {checkpoint['epoch']}")
    if 'best_val_loss' in checkpoint:
        print(f"✓ Best val loss: {checkpoint['best_val_loss']:.4f}")

    return model

def load_test_data(data_path):
    """Load test dataset."""
    print(f"Loading data from: {data_path}")
    with open(data_path, 'r') as f:
        data = json.load(f)

    if 'samples' in data:
        samples = data['samples']
    else:
        samples = data

    return samples

def decode_id(vocab, token_id):
    """Decode token ID to label."""
    id_to_label = {int(k): v for k, v in vocab['id_to_label'].items()}
    return id_to_label.get(token_id, f"UNK_{token_id}")

def evaluate(model, dataloader, vocab_size, device):
    """Evaluate model accuracy."""
    correct_top1 = 0
    correct_top5 = 0
    correct_top10 = 0
    total = 0

    model.eval()
    with torch.no_grad():
        for contexts, targets in tqdm(dataloader, desc="Evaluating"):
            contexts = contexts.to(device)
            targets = targets.to(device)

            # Forward pass
            logits = model(contexts)
            # Get logits for last token: [batch_size, vocab_size]
            next_token_logits = logits[:, -1, :]

            # Top-k predictions
            _, top10_preds = torch.topk(next_token_logits, k=10, dim=1)

            # Check accuracy
            # top10_preds: [batch_size, 10]
            # targets: [batch_size]
            targets_expanded = targets.unsqueeze(1)

            # Check if target is in top K
            # matches: [batch_size, 10] boolean
            matches = (top10_preds == targets_expanded)

            correct_top1 += matches[:, 0].sum().item()
            correct_top5 += matches[:, :5].any(dim=1).sum().item()
            correct_top10 += matches[:, :10].any(dim=1).sum().item()
            total += targets.size(0)

    return {
        'top1': correct_top1 / total,
        'top5': correct_top5 / total,
        'top10': correct_top10 / total,
        'total': total
    }

def main():
    parser = argparse.ArgumentParser(description='Evaluate FOL Transformer')
    parser.add_argument('--checkpoint', type=str, required=True, help='Path to model checkpoint')
    parser.add_argument('--test-data', type=str, required=True, help='Path to test data JSON')
    parser.add_argument('--vocab', type=str, default='unified_vocabulary.json', help='Path to vocabulary')
    parser.add_argument('--batch-size', type=int, default=128, help='Batch size for evaluation')
    parser.add_argument('--no-cuda', action='store_true', help='Disable CUDA')

    args = parser.parse_args()

    # Setup device
    use_cuda = not args.no_cuda and torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    print(f"Using device: {device}")

    # Check files
    if not os.path.exists(args.checkpoint):
        print(f"Error: Checkpoint not found at {args.checkpoint}")
        return
    if not os.path.exists(args.test_data):
        print(f"Error: Test data not found at {args.test_data}")
        return

    # Load vocab
    vocab = load_vocabulary(args.vocab)
    vocab_size = vocab['vocab_size']
    print(f"Vocabulary size: {vocab_size}")

    # Load data
    samples = load_test_data(args.test_data)
    dataset = FOLDataset(samples)
    dataloader = DataLoader(
        dataset,
        batch_size=args.batch_size,
        shuffle=False,
        collate_fn=collate_fn,
        num_workers=2 if use_cuda else 0
    )
    print(f"Test samples: {len(samples)}")

    # Load model
    model = load_model_checkpoint(args.checkpoint, vocab_size, device)

    # Evaluate
    print("\n" + "="*50)
    print("STARTING EVALUATION")
    print("="*50)

    metrics = evaluate(model, dataloader, vocab_size, device)

    print("\n" + "="*50)
    print("RESULTS")
    print("="*50)
    print(f"Top-1 Accuracy:  {metrics['top1']:.2%} ({metrics['top1']*100:.1f}%)")
    print(f"Top-5 Accuracy:  {metrics['top5']:.2%} ({metrics['top5']*100:.1f}%)")
    print(f"Top-10 Accuracy: {metrics['top10']:.2%} ({metrics['top10']*100:.1f}%)")

    # Random baseline comparison
    random_acc = 1.0 / vocab_size
    improvement = metrics['top1'] / random_acc
    print(f"\nImprovement over random ({random_acc:.2%}): {improvement:.1f}x")

if __name__ == '__main__':
    main()


Overwriting evaluate_model.py


**Reasoning**:
Rewrite evaluate_model.py to implement batch processing and argument parsing as requested.



In [ ]:
%%writefile evaluate_model.py
#!/usr/bin/env python3
"""
Evaluate trained FOL model with batch processing.
Optimized for speed using DataLoader.
"""

import os
import sys
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from pathlib import Path
import json
import argparse
from tqdm import tqdm

# Add fol_transformer to path to ensure imports work
sys.path.insert(0, os.path.join(os.path.dirname(__file__), 'fol_transformer'))

class FOLDataset(Dataset):
    """Dataset for FOL formulas."""
    def __init__(self, samples):
        self.samples = samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        # Handle different formats
        if 'input_ids' in sample:
            context = sample['input_ids']
            target = sample['target']
        elif 'context' in sample:
            context = sample['context']
            target = sample['target']
        elif 'input' in sample and 'target' in sample:
            context = sample['input']
            target = sample['target']
        else:
            # Fallback or error
            context = []
            target = 0

        return torch.tensor(context, dtype=torch.long), torch.tensor(target, dtype=torch.long)

def collate_fn(batch):
    """Pad sequences in batch."""
    contexts, targets = zip(*batch)
    # Pad contexts to max length in this batch
    padded_contexts = pad_sequence(contexts, batch_first=True, padding_value=0)
    targets = torch.stack(targets)
    return padded_contexts, targets

def load_vocabulary(vocab_path):
    """Load vocabulary."""
    with open(vocab_path, 'r', encoding='utf-8') as f:
        vocab = json.load(f)
    return vocab

def load_model_checkpoint(checkpoint_path, vocab_size, device='cpu'):
    """Load trained model from checkpoint."""
    # Import here to avoid issues if paths aren't set up at module level
    from models.transformer import create_model

    print(f"Loading checkpoint: {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path, map_location=device)
    config = checkpoint['config']

    # Create model
    model = create_model(vocab_size, config['model_size'])
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    model.eval()

    print(f"✓ Loaded model from epoch {checkpoint['epoch']}")
    if 'best_val_loss' in checkpoint:
        print(f"✓ Best val loss: {checkpoint['best_val_loss']:.4f}")

    return model

def load_test_data(data_path):
    """Load test dataset."""
    print(f"Loading data from: {data_path}")
    with open(data_path, 'r') as f:
        data = json.load(f)

    if 'samples' in data:
        samples = data['samples']
    else:
        samples = data

    return samples

def decode_id(vocab, token_id):
    """Decode token ID to label."""
    id_to_label = {int(k): v for k, v in vocab['id_to_label'].items()}
    return id_to_label.get(token_id, f"UNK_{token_id}")

def evaluate(model, dataloader, vocab_size, device):
    """Evaluate model accuracy."""
    correct_top1 = 0
    correct_top5 = 0
    correct_top10 = 0
    total = 0

    model.eval()
    with torch.no_grad():
        for contexts, targets in tqdm(dataloader, desc="Evaluating"):
            contexts = contexts.to(device)
            targets = targets.to(device)

            # Forward pass
            logits = model(contexts)
            # Get logits for last token: [batch_size, vocab_size]
            next_token_logits = logits[:, -1, :]

            # Top-k predictions
            _, top10_preds = torch.topk(next_token_logits, k=10, dim=1)

            # Check accuracy
            # top10_preds: [batch_size, 10]
            # targets: [batch_size]
            targets_expanded = targets.unsqueeze(1)

            # Check if target is in top K
            # matches: [batch_size, 10] boolean
            matches = (top10_preds == targets_expanded)

            correct_top1 += matches[:, 0].sum().item()
            correct_top5 += matches[:, :5].any(dim=1).sum().item()
            correct_top10 += matches[:, :10].any(dim=1).sum().item()
            total += targets.size(0)

    return {
        'top1': correct_top1 / total,
        'top5': correct_top5 / total,
        'top10': correct_top10 / total,
        'total': total
    }

def main():
    parser = argparse.ArgumentParser(description='Evaluate FOL Transformer')
    parser.add_argument('--checkpoint', type=str, required=True, help='Path to model checkpoint')
    parser.add_argument('--test-data', type=str, required=True, help='Path to test data JSON')
    parser.add_argument('--vocab', type=str, default='unified_vocabulary.json', help='Path to vocabulary')
    parser.add_argument('--batch-size', type=int, default=128, help='Batch size for evaluation')
    parser.add_argument('--no-cuda', action='store_true', help='Disable CUDA')

    args = parser.parse_args()

    # Setup device
    use_cuda = not args.no_cuda and torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    print(f"Using device: {device}")

    # Check files
    if not os.path.exists(args.checkpoint):
        print(f"Error: Checkpoint not found at {args.checkpoint}")
        return
    if not os.path.exists(args.test_data):
        print(f"Error: Test data not found at {args.test_data}")
        return

    # Load vocab
    vocab = load_vocabulary(args.vocab)
    vocab_size = vocab['vocab_size']
    print(f"Vocabulary size: {vocab_size}")

    # Load data
    samples = load_test_data(args.test_data)
    dataset = FOLDataset(samples)
    dataloader = DataLoader(
        dataset,
        batch_size=args.batch_size,
        shuffle=False,
        collate_fn=collate_fn,
        num_workers=2 if use_cuda else 0
    )
    print(f"Test samples: {len(samples)}")

    # Load model
    model = load_model_checkpoint(args.checkpoint, vocab_size, device)

    # Evaluate
    print("\n" + "="*50)
    print("STARTING EVALUATION")
    print("="*50)

    metrics = evaluate(model, dataloader, vocab_size, device)

    print("\n" + "="*50)
    print("RESULTS")
    print("="*50)
    print(f"Top-1 Accuracy:  {metrics['top1']:.2%} ({metrics['top1']*100:.1f}%)")
    print(f"Top-5 Accuracy:  {metrics['top5']:.2%} ({metrics['top5']*100:.1f}%)")
    print(f"Top-10 Accuracy: {metrics['top10']:.2%} ({metrics['top10']*100:.1f}%)")

    # Random baseline comparison
    random_acc = 1.0 / vocab_size
    improvement = metrics['top1'] / random_acc
    print(f"\nImprovement over random ({random_acc:.2%}): {improvement:.1f}x")

if __name__ == '__main__':
    main()


Overwriting evaluate_model.py


## Run Evaluation with Config

### Subtask:
Execute the optimized evaluation script with the specified batch size.


**Reasoning**:
Execute the evaluation script with the specified parameters to assess model performance using the optimized batch processing.



In [ ]:
import os

os.chdir('/content/Symbolic-Transformers')

print("📊 Running optimized evaluation...")
!python evaluate_model.py \
    --checkpoint checkpoints/best_model.pt \
    --test-data datasets/fol_next_symbol/test.json \
    --batch-size 32

📊 Running optimized evaluation...
Using device: cuda
Vocabulary size: 662
Loading data from: datasets/fol_next_symbol/test.json
Test samples: 81694
Loading checkpoint: checkpoints/best_model.pt
✓ Created tiny model with 566,934 parameters
✓ Loaded model from epoch 85
✓ Best val loss: 0.8813

STARTING EVALUATION
Evaluating: 100% 2553/2553 [00:07<00:00, 324.41it/s]

RESULTS
Top-1 Accuracy:  56.86% (56.9%)
Top-5 Accuracy:  88.67% (88.7%)
Top-10 Accuracy: 94.70% (94.7%)

Improvement over random (0.15%): 376.4x


## Summary:

### Data Analysis Key Findings
*   The optimized evaluation script successfully processed **81,694** test samples using batch processing (batch size: 32).
*   The model, loaded from Epoch 85 (best validation loss: 0.8813), achieved a **Top-1 Accuracy of 56.86%**, correctly predicting the exact next symbol in the sequence more than half the time.
*   Broader prediction metrics showed high reliability, with a **Top-5 Accuracy of 88.67%** and a **Top-10 Accuracy of 94.70%**.
*   The model's performance represents a **376.4x improvement** over a random guessing baseline (approx. 0.15% accuracy given the vocabulary size of 662).

### Insights or Next Steps
*   The significant gap between Top-1 (56.86%) and Top-5 (88.67%) accuracy indicates that the model is highly effective at narrowing down the context to a few plausible symbols, even if it doesn't always rank the exact target highest.
*   The successful implementation of `DataLoader` has optimized the evaluation pipeline; future steps should focus on qualitative error analysis to understand the specific ambiguity causing the Top-1 misses.


# Task
Run the interactive demo to qualitatively verify the model's predictions on sample First-Order Logic formulas.

# Plan:
* **Create Demo Script**: Create a Python script `demo.py` containing the logic from the "Quick Demo" cell (Cell 7) to load the trained model and run inference on specific examples.
* **Run Demo Script**: Execute `demo.py` to see the model's top-5 predictions for various input contexts.
* **Final Task**: Confirm that the model correctly predicts the next tokens for the given FOL sequences.

## Create Demo Script

### Subtask:
Create a Python script `demo.py` containing the logic from the "Quick Demo" cell to load the trained model and run inference.


**Reasoning**:
Create the demo.py script in the specified directory to fulfill the subtask requirements.



In [ ]:
%%writefile demo.py
import torch
import sys
import os

# Ensure we can import from the current directory
sys.path.insert(0, os.getcwd())

from utils.vocabulary import Vocabulary
from models.transformer import SymbolicTransformer, get_model_config

def main():
    print("🔄 Loading model...")

    # Load Vocabulary
    if not os.path.exists('unified_vocabulary.json'):
        print("❌ Error: unified_vocabulary.json not found.")
        return

    vocab = Vocabulary('unified_vocabulary.json')

    # Setup Device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: {device}")

    # Load Checkpoint
    checkpoint_path = 'checkpoints/best_model.pt'
    if not os.path.exists(checkpoint_path):
        print(f"❌ Error: Checkpoint not found at {checkpoint_path}")
        return

    # Load model state
    # weights_only=False is used to allow loading the arbitrary config dict
    checkpoint = torch.load(checkpoint_path, map_location=device)

    # Initialize Model
    config = checkpoint['config']
    model_config = get_model_config(config['model_size'], vocab.vocab_size)
    model = SymbolicTransformer(**model_config).to(device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    print(f"✓ Loaded {config['model_size']} model from epoch {checkpoint['epoch']}")
    if 'best_val_loss' in checkpoint:
        print(f"  Val loss: {checkpoint['best_val_loss']:.4f}\n")

    # Prediction Helper
    def predict_next(tokens, top_k=5):
        """Predict next token given a sequence."""
        # Convert tokens to IDs
        # Logic handles both symbolic labels (in vocab) and direct integer tokens (numerals)
        token_ids = []
        for t in tokens:
            if t in vocab.label_to_id:
                token_ids.append(vocab.encode_label(t))
            else:
                try:
                    token_ids.append(int(t))
                except ValueError:
                    print(f"Warning: Skipping unknown token '{t}'")

        x = torch.tensor([token_ids], device=device, dtype=torch.long)

        with torch.no_grad():
            logits = model(x)
            # Get probabilities for the last token position
            probs = torch.softmax(logits[0, -1], dim=-1)
            top_probs, top_ids = torch.topk(probs, top_k)

        print(f"Input: {' '.join(str(t) for t in tokens)}")
        print(f"Top {top_k} predictions:")
        for i, (prob, tid) in enumerate(zip(top_probs, top_ids)):
            label = vocab.decode_id(tid.item())
            bar = '█' * int(prob * 30)
            print(f"  {i+1}. {label:12s} {prob*100:5.1f}% {bar}")
        print()

    # Run Samples
    print("="*50)
    print("📊 EXAMPLE PREDICTIONS")
    print("="*50 + "\n")

    predict_next(['FORALL'])
    predict_next(['FORALL', 'VAR'])
    predict_next(['FORALL', 'VAR', '1'])
    predict_next(['PRED', '3'])

    print("\n💡 The model learned FOL syntax rules!")

if __name__ == "__main__":
    main()

Writing demo.py


**Reasoning**:
Run the newly created demo script to verify the model's interactive predictions.



In [ ]:
import os

os.chdir('/content/Symbolic-Transformers')

print("🚀 Running interactive demo script...")
!python demo.py

🚀 Running interactive demo script...
Traceback (most recent call last):
  File "/content/Symbolic-Transformers/demo.py", line 9, in <module>
    from models.transformer import SymbolicTransformer, get_model_config
ImportError: cannot import name 'get_model_config' from 'models.transformer' (/content/Symbolic-Transformers/models/transformer.py)


**Reasoning**:
The previous run failed because `get_model_config` could not be imported. I need to inspect `models/transformer.py` to identify the correct function or class for model configuration.



In [ ]:
import os
os.chdir('/content/Symbolic-Transformers')
!cat models/transformer.py

"""
Symbolic Transformer Model for FOL Next-Symbol Prediction.

Optimized for AMD Radeon GPU (ROCm backend).
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from typing import Optional, Tuple


class PositionalEncoding(nn.Module):
    """
    Sinusoidal positional encoding for transformer.
    Standard implementation from "Attention is All You Need".
    """
    
    def __init__(self, d_model: int, max_len: int = 512, dropout: float = 0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Create positional encoding matrix
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * 
                            (-math.log(10000.0) / d_model))
        
        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        
        # Register as buffer (not a parameter)
     

**Reasoning**:
The previous execution failed because 'get_model_config' does not exist in 'models/transformer.py'. I will modify 'demo.py' to use the 'create_model' function which exists in the module, instead of trying to manually configure the SymbolicTransformer.



In [ ]:
%%writefile demo.py
import torch
import sys
import os

# Ensure we can import from the current directory
sys.path.insert(0, os.getcwd())

from utils.vocabulary import Vocabulary
from models.transformer import create_model

def main():
    print("🔄 Loading model...")

    # Load Vocabulary
    if not os.path.exists('unified_vocabulary.json'):
        print("❌ Error: unified_vocabulary.json not found.")
        return

    vocab = Vocabulary('unified_vocabulary.json')

    # Setup Device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Device: {device}")

    # Load Checkpoint
    checkpoint_path = 'checkpoints/best_model.pt'
    if not os.path.exists(checkpoint_path):
        print(f"❌ Error: Checkpoint not found at {checkpoint_path}")
        return

    # Load model state
    checkpoint = torch.load(checkpoint_path, map_location=device)
    config = checkpoint['config']

    # Initialize Model using create_model factory function
    try:
        model = create_model(vocab_size=vocab.vocab_size, model_size=config['model_size'])
        model = model.to(device)
        model.load_state_dict(checkpoint['model_state_dict'])
        model.eval()
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        return

    print(f"✓ Loaded {config['model_size']} model from epoch {checkpoint['epoch']}")
    if 'best_val_loss' in checkpoint:
        print(f"  Val loss: {checkpoint['best_val_loss']:.4f}\n")

    # Prediction Helper
    def predict_next(tokens, top_k=5):
        """Predict next token given a sequence."""
        # Convert tokens to IDs
        token_ids = []
        for t in tokens:
            if t in vocab.label_to_id:
                token_ids.append(vocab.encode_label(t))
            else:
                try:
                    token_ids.append(int(t))
                except ValueError:
                    print(f"Warning: Skipping unknown token '{t}'")

        if not token_ids:
            print("Error: No valid tokens found.")
            return

        x = torch.tensor([token_ids], device=device, dtype=torch.long)

        with torch.no_grad():
            logits = model(x)
            # Get probabilities for the last token position
            probs = torch.softmax(logits[0, -1], dim=-1)
            top_probs, top_ids = torch.topk(probs, top_k)

        print(f"Input: {' '.join(str(t) for t in tokens)}")
        print(f"Top {top_k} predictions:")
        for i, (prob, tid) in enumerate(zip(top_probs, top_ids)):
            label = vocab.decode_id(tid.item())
            bar = '█' * int(prob * 30)
            print(f"  {i+1}. {label:12s} {prob*100:5.1f}% {bar}")
        print()

    # Run Samples
    print("="*50)
    print("📊 EXAMPLE PREDICTIONS")
    print("="*50 + "\n")

    predict_next(['FORALL'])
    predict_next(['FORALL', 'VAR'])
    predict_next(['FORALL', 'VAR', '1'])
    predict_next(['PRED', '3'])

    print("\n💡 The model learned FOL syntax rules!")

if __name__ == "__main__":
    main()

Overwriting demo.py


**Reasoning**:
Execute the corrected demo script to verify the interactive predictions.



In [ ]:
import os

os.chdir('/content/Symbolic-Transformers')

print("🚀 Running fixed interactive demo script...")
!python demo.py

🚀 Running fixed interactive demo script...
🔄 Loading model...
✓ Vocabulary loaded: 662 tokens
  - Numerals: 0-624
  - Symbols: 625-661
  - Compositional: ['VAR', 'CONST', 'PRED', 'FUNC', 'SORT']
Device: cuda
✓ Created tiny model with 566,934 parameters
✓ Loaded tiny model from epoch 85
  Val loss: 0.8813

📊 EXAMPLE PREDICTIONS

Input: FORALL
Top 5 predictions:
  1. VAR           99.8% █████████████████████████████
  2. FUNC           0.1% 
  3. CONST          0.1% 
  4. PRED           0.0% 
  5. COMMA          0.0% 

Input: FORALL VAR
Top 5 predictions:
  1. VAR           94.3% ████████████████████████████
  2. FUNC           3.8% █
  3. CONST          1.9% 
  4. PRED           0.0% 
  5. NUM_003        0.0% 

Input: FORALL VAR 1
Top 5 predictions:
  1. VAR           73.2% █████████████████████
  2. FUNC          21.8% ██████
  3. CONST          2.8% 
  4. LPAREN         0.5% 
  5. PRED           0.5% 

Input: PRED 3
Top 5 predictions:
  1. IMPLIES       51.8% ███████████████
  2. CONS

## Modify Training Script for Batch Size

### Subtask:
Read the `training/train.py` file to understand the current argument parsing logic.


**Reasoning**:
Read the content of training/train.py to understand how to add the batch-size argument.



In [ ]:
import os

os.chdir('/content/Symbolic-Transformers')
!cat training/train.py

"""
Training script for Symbolic FOL Transformer.
Optimized for AMD Radeon GPU with ROCm backend.
"""

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import json
import time
from pathlib import Path
from typing import Dict, Optional, List
from dataclasses import dataclass, asdict
import sys
import argparse
import re

sys.path.append(str(Path(__file__).parent.parent))

from models.transformer import create_model
from utils.vocabulary import Vocabulary


def get_training_config(model_size='tiny', num_epochs=50, batch_size=None):
    """Helper to get training config for different model sizes."""

    # Auto-adjust batch size based on model
    if batch_size is None:
        batch_size = {
            'tiny': 64,
            'small': 32,
            'base': 16,
            'large': 8,
        }[model_size]

    return TrainingConfig(
        model_size=model_size,
        batch_size=batch_size,
        num_epochs=num_epochs,


**Reasoning**:
Modify training/train.py to include the --batch-size argument in the ArgumentParser and pass it to get_training_config.



In [ ]:
%%writefile training/train.py
"""
Training script for Symbolic FOL Transformer.
Optimized for AMD Radeon GPU with ROCm backend.
"""

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import json
import time
from pathlib import Path
from typing import Dict, Optional, List
from dataclasses import dataclass, asdict
import sys
import argparse
import re

sys.path.append(str(Path(__file__).parent.parent))

from models.transformer import create_model
from utils.vocabulary import Vocabulary


def get_training_config(model_size='tiny', num_epochs=50, batch_size=None):
    """Helper to get training config for different model sizes."""

    # Auto-adjust batch size based on model
    if batch_size is None:
        batch_size = {
            'tiny': 64,
            'small': 32,
            'base': 16,
            'large': 8,
        }[model_size]

    return TrainingConfig(
        model_size=model_size,
        batch_size=batch_size,
        num_epochs=num_epochs,
        learning_rate=1e-4,
        warmup_steps=2000,
        checkpoint_dir='checkpoints',
    )


@dataclass
class TrainingConfig:
    """Training configuration."""
    # Model
    model_size: str = 'base'
    vocab_size: int = 663

    # Training
    batch_size: int = 64  # Larger batches for larger dataset
    num_epochs: int = 50
    learning_rate: float = 1e-4
    weight_decay: float = 0.01
    warmup_steps: int = 2000  # More warmup for larger dataset
    max_grad_norm: float = 1.0

    # Data
    max_seq_len: int = 128
    train_data_path: str = "datasets/fol_next_symbol/train.json"
    val_data_path: str = "datasets/fol_next_symbol/val.json"

    # Checkpointing
    checkpoint_dir: str = "checkpoints"
    save_every: int = 5  # Save every N epochs

    # Logging
    log_every: int = 100  # Log every N batches

    # Device
    device: str = "cuda"  # Will use ROCm if available
    mixed_precision: bool = False  # AMD GPU may not support all AMP ops


class FOLDataset(Dataset):
    """PyTorch Dataset for FOL next-symbol prediction."""

    def __init__(self, data_path: str, max_seq_len: int = 128):
        with open(data_path, 'r') as f:
            data = json.load(f)

        self.samples = data['samples']
        self.max_seq_len = max_seq_len

        print(f"✓ Loaded {len(self.samples)} samples from {data_path}")

    def __len__(self) -> int:
        return len(self.samples)

    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        sample = self.samples[idx]

        # Get context and target
        context = sample['context']
        target = sample['target']

        # Pad context to max_seq_len
        context_len = len(context)
        if context_len < self.max_seq_len:
            context = context + [0] * (self.max_seq_len - context_len)
        else:
            context = context[:self.max_seq_len]
            context_len = self.max_seq_len

        return {
            'input_ids': torch.tensor(context, dtype=torch.long),
            'target': torch.tensor(target, dtype=torch.long),
            'length': torch.tensor(context_len, dtype=torch.long)
        }


class Trainer:
    """Trainer for Symbolic Transformer."""

    def __init__(
        self,
        model: nn.Module,
        train_loader: DataLoader,
        val_loader: DataLoader,
        config: TrainingConfig,
        vocab: Vocabulary
    ):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.config = config
        self.vocab = vocab

        # Setup device
        self.device = torch.device(config.device if torch.cuda.is_available() else "cpu")
        print(f"✓ Using device: {self.device}")

        if torch.cuda.is_available():
            print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
            print(f"✓ VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

        self.model = self.model.to(self.device)

        # Setup optimizer
        self.optimizer = optim.AdamW(
            model.parameters(),
            lr=config.learning_rate,
            weight_decay=config.weight_decay,
            betas=(0.9, 0.98),
            eps=1e-9
        )

        # Learning rate scheduler with warmup
        def lr_lambda(step):
            if step < config.warmup_steps:
                return step / config.warmup_steps
            return max(0.1, 0.5 * (1.0 + torch.cos(torch.tensor(
                (step - config.warmup_steps) / (len(train_loader) * config.num_epochs)
            ) * 3.14159)))

        self.scheduler = optim.lr_scheduler.LambdaLR(self.optimizer, lr_lambda)

        # Loss function
        self.criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignore padding

        # Training state
        self.global_step = 0
        self.epoch = 0
        self.best_val_loss = float('inf')

        # Setup checkpoint directory
        self.checkpoint_dir = Path(config.checkpoint_dir)
        self.checkpoint_dir.mkdir(parents=True, exist_ok=True)

        # Training history
        self.history = {
            'train_loss': [],
            'val_loss': [],
            'learning_rate': []
        }

    def train_epoch(self) -> float:
        """Train for one epoch."""
        self.model.train()
        total_loss = 0
        num_batches = 0

        for batch_idx, batch in enumerate(self.train_loader):
            # Move to device
            input_ids = batch['input_ids'].to(self.device)
            targets = batch['target'].to(self.device)

            # Forward pass
            logits = self.model(input_ids)

            # Compute loss on last position (next-token prediction)
            loss = self.criterion(logits[:, -1, :], targets)

            # Backward pass
            self.optimizer.zero_grad()
            loss.backward()

            # Clip gradients
            torch.nn.utils.clip_grad_norm_(
                self.model.parameters(),
                self.config.max_grad_norm
            )

            # Update weights
            self.optimizer.step()
            self.scheduler.step()

            # Track metrics
            total_loss += loss.item()
            num_batches += 1
            self.global_step += 1

            # Logging
            if (batch_idx + 1) % self.config.log_every == 0:
                avg_loss = total_loss / num_batches
                lr = self.scheduler.get_last_lr()[0]

                print(f"  Batch {batch_idx + 1}/{len(self.train_loader)} | "
                      f"Loss: {avg_loss:.4f} | LR: {lr:.2e}")

        return total_loss / num_batches

    def validate(self) -> float:
        """Run validation."""
        self.model.eval()
        total_loss = 0
        num_batches = 0

        with torch.no_grad():
            for batch in self.val_loader:
                input_ids = batch['input_ids'].to(self.device)
                targets = batch['target'].to(self.device)

                logits = self.model(input_ids)
                loss = self.criterion(logits[:, -1, :], targets)

                total_loss += loss.item()
                num_batches += 1

        return total_loss / num_batches

    def save_checkpoint(self, is_best: bool = False):
        """Save model checkpoint."""
        checkpoint = {
            'epoch': self.epoch,
            'global_step': self.global_step,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'best_val_loss': self.best_val_loss,
            'config': asdict(self.config),
            'history': self.history
        }

        # Save regular checkpoint
        checkpoint_path = self.checkpoint_dir / f"checkpoint_epoch_{self.epoch}.pt"
        torch.save(checkpoint, checkpoint_path)
        print(f"  ✓ Saved checkpoint: {checkpoint_path}")

        # Save best model
        if is_best:
            best_path = self.checkpoint_dir / "best_model.pt"
            torch.save(checkpoint, best_path)
            print(f"  ✓ Saved best model: {best_path}")

    def load_checkpoint(self, checkpoint_path: str):
        """Load model and optimizer state from a checkpoint."""
        checkpoint = torch.load(checkpoint_path, map_location=self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        self.epoch = checkpoint.get('epoch', 0)
        self.global_step = checkpoint.get('global_step', 0)
        self.best_val_loss = checkpoint.get('best_val_loss', float('inf'))
        self.history = checkpoint.get('history', self.history)
        print(f"✓ Resumed from checkpoint: {checkpoint_path}")

    def train(self, start_epoch: int = 0):
        """Main training loop."""
        print("\n" + "="*60)
        print("TRAINING START")
        print("="*60)
        print(f"Model: {self.config.model_size}")
        print(f"Vocab size: {self.config.vocab_size}")
        print(f"Batch size: {self.config.batch_size}")
        print(f"Epochs: {self.config.num_epochs}")
        print(f"Device: {self.device}")
        print("="*60 + "\n")

        for epoch in range(start_epoch, self.config.num_epochs):
            self.epoch = epoch + 1
            start_time = time.time()

            print(f"Epoch {self.epoch}/{self.config.num_epochs}")

            # Train
            train_loss = self.train_epoch()

            # Validate
            val_loss = self.validate()

            # Track history
            self.history['train_loss'].append(train_loss)
            self.history['val_loss'].append(val_loss)
            self.history['learning_rate'].append(self.scheduler.get_last_lr()[0])

            # Check if best model
            is_best = val_loss < self.best_val_loss
            if is_best:
                self.best_val_loss = val_loss

            # Print epoch summary
            epoch_time = time.time() - start_time
            print(f"\n  Train Loss: {train_loss:.4f}")
            print(f"  Val Loss:   {val_loss:.4f} {'[BEST]' if is_best else ''}")
            print(f"  Time:       {epoch_time:.1f}s\n")

            # Save checkpoint
            if self.epoch % self.config.save_every == 0 or is_best:
                self.save_checkpoint(is_best)

        print("="*60)
        print("TRAINING COMPLETE")
        print("="*60)
        print(f"Best validation loss: {self.best_val_loss:.4f}")
        print(f"Final checkpoint: {self.checkpoint_dir / f'checkpoint_epoch_{self.epoch}.pt'}")
        print("="*60)


def main():
    """Main training function."""
    parser = argparse.ArgumentParser(description="Train the Symbolic FOL Transformer")
    parser.add_argument(
        "--resume",
        nargs="?",
        const="latest",
        help="Resume from checkpoint path or 'latest' in checkpoint_dir"
    )
    parser.add_argument(
        "--num-epochs",
        type=int,
        help="Total number of epochs to train (overrides config)"
    )
    parser.add_argument(
        "--batch-size",
        type=int,
        help="Batch size (overrides config)"
    )
    parser.add_argument(
        "--model-size",
        default="tiny",
        choices=["tiny", "small", "base", "large"],
        help="Model size preset"
    )
    args = parser.parse_args()

    def resolve_checkpoint_path(resume_arg: str, checkpoint_dir: Path) -> str:
        if resume_arg == "latest":
            checkpoints = list(checkpoint_dir.glob("checkpoint_epoch_*.pt"))
            if not checkpoints:
                raise FileNotFoundError(f"No checkpoints found in {checkpoint_dir}")
            def checkpoint_epoch(path: Path) -> int:
                match = re.search(r"checkpoint_epoch_(\d+)\.pt$", path.name)
                return int(match.group(1)) if match else -1
            latest_checkpoint = max(checkpoints, key=checkpoint_epoch)
            return str(latest_checkpoint)
        return resume_arg

    # Configuration
    config = get_training_config(
        model_size=args.model_size,
        num_epochs=args.num_epochs or 50,
        batch_size=args.batch_size,
    )

    # Load vocabulary
    vocab_path = "unified_vocabulary.json"
    vocab = Vocabulary(vocab_path)
    config.vocab_size = vocab.vocab_size

    print(f"\n✓ Loaded vocabulary: {vocab.vocab_size} tokens")

    resume_path: Optional[str] = None
    if args.resume:
        resume_path = resolve_checkpoint_path(args.resume, Path(config.checkpoint_dir))
        checkpoint_meta = torch.load(resume_path, map_location="cpu")
        ckpt_config = checkpoint_meta.get('config', {})
        if ckpt_config.get('model_size'):
            config.model_size = ckpt_config['model_size']
        if ckpt_config.get('vocab_size'):
            config.vocab_size = ckpt_config['vocab_size']
        if ckpt_config.get('max_seq_len'):
            config.max_seq_len = ckpt_config['max_seq_len']
        if ckpt_config.get('num_epochs') and args.num_epochs is None:
            config.num_epochs = ckpt_config['num_epochs']

    if args.num_epochs is not None:
        config.num_epochs = args.num_epochs

    # Create datasets
    print("\nLoading datasets...")
    train_dataset = FOLDataset(config.train_data_path, config.max_seq_len)
    val_dataset = FOLDataset(config.val_data_path, config.max_seq_len)

    # Create dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=config.batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=config.batch_size,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )

    print(f"✓ Train batches: {len(train_loader)}")
    print(f"✓ Val batches: {len(val_loader)}")

    # Create model
    print("\nCreating model...")
    model = create_model(config.vocab_size, config.model_size)

    # Create trainer
    trainer = Trainer(model, train_loader, val_loader, config, vocab)

    start_epoch = 0
    if resume_path:
        trainer.load_checkpoint(resume_path)
        start_epoch = trainer.epoch

    # Train
    trainer.train(start_epoch=start_epoch)


if __name__ == "__main__":
    main()


Overwriting training/train.py


## Verify and Run Training

### Subtask:
Verify training/train.py updates and execute training with the configured batch size.


**Reasoning**:
Verify that `training/train.py` has been updated to accept the `--batch-size` argument and then execute the training command using the configuration from the previous steps.



In [ ]:
import os

# Change directory to the repository
os.chdir('/content/Symbolic-Transformers')

# 1. Verify that training/train.py accepts a --batch-size argument
print("🔍 Verifying training/train.py...")
with open('training/train.py', 'r') as f:
    content = f.read()
    if "--batch-size" in content:
        print("✓ verified: training/train.py accepts --batch-size")
    else:
        print("⚠️ Warning: --batch-size not found in training/train.py")

# 2. Construct the training command
print("\n🚀 Starting Training...")
print(f"Model: {config['model_size']} | Epochs: {config['num_epochs']} | Batch Size: {config['batch_size']}")
if config['resume']:
    print("Resuming from last checkpoint...")
print("="*60 + "\n")

# Build training command
cmd = f"python training/train.py --model-size {config['model_size']} --num-epochs {config['num_epochs']} --batch-size {config['batch_size']}"
if config['resume']:
    cmd += " --resume"

# 3. Run training
!{cmd}

🔍 Verifying training/train.py...
✓ verified: training/train.py accepts --batch-size

🚀 Starting Training...
Model: tiny | Epochs: 500 | Batch Size: 32

✓ Vocabulary loaded: 662 tokens
  - Numerals: 0-624
  - Symbols: 625-661
  - Compositional: ['VAR', 'CONST', 'PRED', 'FUNC', 'SORT']

✓ Loaded vocabulary: 662 tokens

Loading datasets...
✓ Loaded 1682302 samples from datasets/fol_next_symbol/train.json
✓ Loaded 166031 samples from datasets/fol_next_symbol/val.json
✓ Train batches: 52572
✓ Val batches: 5189

Creating model...
✓ Created tiny model with 566,934 parameters
✓ Using device: cuda
✓ GPU: NVIDIA A100-SXM4-80GB
✓ VRAM: 85.2 GB

TRAINING START
Model: tiny
Vocab size: 662
Batch size: 32
Epochs: 500
Device: cuda

Epoch 1/500
  Batch 100/52572 | Loss: 6.5041 | LR: 5.00e-06
  Batch 200/52572 | Loss: 6.3978 | LR: 1.00e-05
  Batch 300/52572 | Loss: 6.2337 | LR: 1.50e-05
  Batch 400/52572 | Loss: 6.0493 | LR: 2.00e-05
  Batch 500/52572 | Loss: 5.8616 | LR: 2.50e-05
  Batch 600/52572 | Lo

# Task
Create a Python script to verify the dataset size and calculate the expected number of batches. Load the `datasets/fol_next_symbol/train.json` file, count the total number of training samples, and divide this count by the batch size of 32. Finally, print the total sample count, the calculated expected batches, and a confirmation message comparing it to the observed 52,572 batches from the training logs.

```python
import json
import math

# Path to the training dataset
train_data_path = "datasets/fol_next_symbol/train.json"

# Load the dataset
print(f"📂 Loading {train_data_path}...")
with open(train_data_path, 'r') as f:
    data = json.load(f)

# Count samples
if 'samples' in data:
    samples = data['samples']
else:
    samples = data
    
total_samples = len(samples)
batch_size = 32

# Calculate expected batches
expected_batches = math.ceil(total_samples / batch_size)

# Display results
print("-" * 50)
print(f"📊 Dataset Analysis")
print("-" * 50)
print(f"Total Training Samples: {total_samples:,}")
print(f"Batch Size:             {batch_size}")
print(f"Expected Batches:       {total_samples:,} / {batch_size} = {expected_batches:,}")
print("-" * 50)

# Verification
observed_batches = 52572
if expected_batches == observed_batches:
    print(f"✅ VERIFIED: The calculated batch count ({expected_batches:,}) matches exactly")
    print(f"   with the training logs ({observed_batches:,}).")
    print(f"   This confirms the model is training with a batch size of {batch_size}.")
else:
    print(f"❌ MISMATCH: Expected {expected_batches:,} but saw {observed_batches:,} in logs.")

print("-" * 50)
print("💡 Insight: The high number of batches is due to the large dataset size.")
print("   To speed up training, you could increase the batch size (e.g., to 128 or 256),")
print("   which would reduce the total number of batches per epoch.")
```

## Verify Dataset Size

### Subtask:
Create and run a Python script to verify the dataset size and calculate the expected number of batches based on the provided code.


**Reasoning**:
Load the training dataset JSON file to count the total number of samples and calculate the expected number of batches given a batch size of 32, verifying this against the observed training output.



In [ ]:
import json
import math

# Define the path to the training dataset
dataset_path = '/content/Symbolic-Transformers/datasets/fol_next_symbol/train.json'

print(f"Loading dataset from {dataset_path}...")

# Open and load the JSON file
with open(dataset_path, 'r') as f:
    data = json.load(f)

# Get total samples
total_samples = len(data['samples'])

# Calculate expected batches
batch_size = 32
expected_batches = math.ceil(total_samples / batch_size)

# Print results
print(f"Total samples: {total_samples}")
print(f"Batch size: {batch_size}")
print(f"Expected batches: {expected_batches}")

# Verify against observed value
observed_batches = 52572
if expected_batches == observed_batches:
    print(f"\u2705 Verified: Expected batches ({expected_batches}) matches observed batches ({observed_batches}).")
else:
    print(f"\u26a0\ufe0f Mismatch: Expected {expected_batches}, but observed {observed_batches}.")

Loading dataset from /content/Symbolic-Transformers/datasets/fol_next_symbol/train.json...
Total samples: 1682302
Batch size: 32
Expected batches: 52572
✅ Verified: Expected batches (52572) matches observed batches (52572).


## Summary:

### Q&A
**Q:** Does the calculated batch count based on the dataset size match the observed training logs?
**A:** Yes, the calculated expected number of batches (52,572) matches exactly with the observed value from the training logs.

### Data Analysis Key Findings
*   The training dataset (`fol_next_symbol/train.json`) contains a total of **1,682,302** samples.
*   With a configured batch size of **32**, the expected number of batches is calculated as 52,572 (derived from 1,682,302 / 32).
*   The analysis confirms a perfect match between the theoretical expectation and the **52,572** batches observed in the training logs, verifying that the model is processing the entire dataset correctly.

### Insights or Next Steps
*   The high number of batches is a direct result of the large dataset size rather than a system error.
*   To potentially accelerate the training process, consider increasing the batch size (e.g., to 128 or 256) if hardware resources allow, which would significantly reduce the number of optimization steps per epoch.


# Task
Print a final summary confirming the resolution of the reported issues. Specifically, verify that:
1.  The `evaluate_model.py` script was successfully refactored to use `DataLoader` for efficient batch processing and now accepts command-line arguments.
2.  The `demo.py` script was fixed to correctly import `create_model` and generate predictions.
3.  The `training/train.py` script was updated to accept a `--batch-size` argument, allowing for flexible training configurations.
4.  The model evaluation was successfully completed, yielding a Top-1 accuracy of **56.86%** and Top-5 accuracy of **88.67%**.

In [ ]:
import os
from google.colab import files
import zipfile

# Define files to package
files_to_download = [
    'evaluate_model.py',
    'demo.py',
    'training/train.py'
]

# Create zip file
zip_filename = 'updated_scripts.zip'
print(f"📦 Creating {zip_filename}...")

with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for file in files_to_download:
        if os.path.exists(file):
            print(f"  Adding {file}")
            zipf.write(file)
        else:
            print(f"⚠️ Warning: {file} not found")

# Trigger download
files.download(zip_filename)
print("✅ Download started.")

📦 Creating updated_scripts.zip...
  Adding evaluate_model.py
  Adding demo.py
  Adding training/train.py


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Download started.


## Final Task

### Subtask:
Confirm the resolution of the reported issues.


## Summary:

### Q&A
*   **Were the reported issues successfully resolved and verified?**
    Yes, the `evaluate_model.py` refactoring, `demo.py` fixes, and `training/train.py` updates were confirmed. The model evaluation was completed successfully with specific accuracy metrics recorded.

### Data Analysis Key Findings
*   The `evaluate_model.py` script was refactored to implement `DataLoader` for efficient batch processing and updated to accept command-line arguments.
*   The `demo.py` script was corrected to properly import `create_model`, restoring its ability to generate predictions.
*   The `training/train.py` script now accepts a `--batch-size` argument, allowing for flexible configuration during training runs.
*   The model achieved a **Top-1 accuracy of 56.86%** and a **Top-5 accuracy of 88.67%** during the final evaluation.

### Insights or Next Steps
*   The codebase has been stabilized and optimized for batch processing, improving both reliability and usability.
*   With the pipeline functional, future efforts can focus on model architecture changes or hyperparameter tuning to improve the 56.86% Top-1 accuracy.


# Task
Delete checkpoint files above checkpoint_epoch_7 from the 'checkpoints' folder.

## Delete Checkpoint Files

### Subtask:
List all .pt files in the 'checkpoints' directory. For each file, extract the epoch number from its name. Delete any checkpoint file where the epoch number is greater than 7.


**Reasoning**:
Implement the instructions to list checkpoint files, extract epoch numbers, and delete those with an epoch greater than 7.



In [ ]:
import os
import re

# Change the current working directory to /content/Symbolic-Transformers
os.chdir('/content/Symbolic-Transformers')
print(f"Current working directory: {os.getcwd()}")

checkpoint_dir = 'checkpoints'

# Check if the checkpoints directory exists
if not os.path.exists(checkpoint_dir):
    print(f"❌ Checkpoints directory '{checkpoint_dir}' not found. No files to delete.")
else:
    print(f"🔍 Listing .pt files in '{checkpoint_dir}'...")
    deleted_count = 0

    # List all files in the checkpoints directory
    for filename in os.listdir(checkpoint_dir):
        if filename.endswith('.pt'):
            # Extract the epoch number using regex
            match = re.match(r"checkpoint_epoch_(\d+)\.pt", filename)
            if match:
                epoch_num = int(match.group(1))

                # Delete if epoch number is greater than 7
                if epoch_num > 7:
                    file_path = os.path.join(checkpoint_dir, filename)
                    os.remove(file_path)
                    print(f"🗑️ Deleted: {filename} (Epoch {epoch_num} > 7)")
                    deleted_count += 1
            else:
                print(f"ℹ️ Skipping non-epoch checkpoint file: {filename}")

    if deleted_count == 0:
        print("✅ No checkpoint files with epoch number > 7 found for deletion.")
    else:
        print(f"Summary: {deleted_count} checkpoint file(s) deleted.")

# Verify remaining files
print(f"\nRemaining files in '{checkpoint_dir}':")
remaining_files = os.listdir(checkpoint_dir)
if remaining_files:
    for f in remaining_files:
        print(f"- {f}")
else:
    print("(None)")

Current working directory: /content/Symbolic-Transformers
🔍 Listing .pt files in 'checkpoints'...
🗑️ Deleted: checkpoint_epoch_51.pt (Epoch 51 > 7)
🗑️ Deleted: checkpoint_epoch_8.pt (Epoch 8 > 7)
🗑️ Deleted: checkpoint_epoch_42.pt (Epoch 42 > 7)
🗑️ Deleted: checkpoint_epoch_31.pt (Epoch 31 > 7)
🗑️ Deleted: checkpoint_epoch_66.pt (Epoch 66 > 7)
🗑️ Deleted: checkpoint_epoch_61.pt (Epoch 61 > 7)
🗑️ Deleted: checkpoint_epoch_58.pt (Epoch 58 > 7)
🗑️ Deleted: checkpoint_epoch_60.pt (Epoch 60 > 7)
🗑️ Deleted: checkpoint_epoch_12.pt (Epoch 12 > 7)
🗑️ Deleted: checkpoint_epoch_9.pt (Epoch 9 > 7)
🗑️ Deleted: checkpoint_epoch_87.pt (Epoch 87 > 7)
🗑️ Deleted: checkpoint_epoch_65.pt (Epoch 65 > 7)
🗑️ Deleted: checkpoint_epoch_80.pt (Epoch 80 > 7)
🗑️ Deleted: checkpoint_epoch_95.pt (Epoch 95 > 7)
🗑️ Deleted: checkpoint_epoch_34.pt (Epoch 34 > 7)
🗑️ Deleted: checkpoint_epoch_72.pt (Epoch 72 > 7)
🗑️ Deleted: checkpoint_epoch_22.pt (Epoch 22 > 7)
🗑️ Deleted: checkpoint_epoch_75.pt (Epoch 75 > 7)
🗑️ Del

## Final Task

### Subtask:
Confirm that checkpoint files above checkpoint_epoch_7 have been deleted from the 'checkpoints' folder.


## Summary:

### Q&A
Yes, checkpoint files above checkpoint\_epoch\_7 have been successfully deleted from the 'checkpoints' folder.

### Data Analysis Key Findings
*   The script successfully navigated to the `/content/Symbolic-Transformers` directory.
*   A total of 45 checkpoint files with epoch numbers greater than 7 were identified and deleted from the `checkpoints` directory.
*   Files like `best_model.pt` and others not matching the `checkpoint_epoch_N.pt` pattern were correctly skipped during the deletion process.
*   After deletion, the remaining files in the `checkpoints` directory included `checkpoint_epoch_6.pt`, `checkpoint_epoch_2.pt`, `checkpoint_epoch_3.pt`, `checkpoint_epoch_4.pt`, `checkpoint_epoch_5.pt`, `checkpoint_epoch_7.pt`, and `best_model.pt`, confirming that files with epoch numbers greater than 7 were removed.

### Insights or Next Steps
*   Implementing an automated checkpoint management strategy can help save disk space and manage model versions efficiently.
*   Consider integrating this script into a larger training pipeline to automatically prune old checkpoints based on predefined criteria.
